In [29]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import os
import json
from PIL import Image
import pickle

In [30]:
A = np.array(pickle.load(open("D:/A.pickle","rb")))

B= np.array(pickle.load(open("D:/B.pickle","rb")))

In [16]:
#print(A.ndim)

4


In [31]:
IMG_SIZE_1=250
IMG_SIZE_2=250

In [36]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Flatten,Reshape
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import TensorBoard
# Load the pre-trained DenseNet-169 model
base_model = DenseNet169(weights='imagenet', include_top=False, input_shape=(IMG_SIZE_1, IMG_SIZE_2, 3))

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Create a new model
model = Sequential()

# Add the DenseNet-169 base model to the new model
model.add(base_model)

# Add a pooling layer to reduce the spatial dimensions
model.add(Flatten())

# Add the LSTM layer
model.add(Reshape((1, -1)))  # Reshape to (1, flattened_features)
model.add(LSTM(64))

# Add fully connected layers
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))

# Add the output layer
num_classes=6
model.add(Dense(num_classes, activation='softmax'))

# Define the optimizer
optimizer = SGD(learning_rate=0.001)

# Compile the model
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
#model.summary()

# Create a TensorBoard callback
NAME = "TIG_DenseNet169-LSTM_model_"

tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

# Train the model with TensorBoard callback
model.fit(A, B, batch_size=10, epochs=5, validation_split=0.2, callbacks=[tensorboard])
    
# Save the model
model.save(f'DenseNet169-LSTM.h5')

Epoch 1/5
2134/2134 [==============================] - 2552s 1s/step - loss: 0.4570 - accuracy: 0.9104 - val_loss: 0.1150 - val_accuracy: 0.9880
Epoch 2/5
2134/2134 [==============================] - 2309s 1s/step - loss: 0.0790 - accuracy: 0.9886 - val_loss: 0.0395 - val_accuracy: 0.9944
Epoch 3/5
2134/2134 [==============================] - 2324s 1s/step - loss: 0.0450 - accuracy: 0.9923 - val_loss: 0.0288 - val_accuracy: 0.9972
Epoch 4/5
2134/2134 [==============================] - 2215s 1s/step - loss: 0.0339 - accuracy: 0.9942 - val_loss: 0.0247 - val_accuracy: 0.9953
Epoch 5/5
2134/2134 [==============================] - 2168s 1s/step - loss: 0.0253 - accuracy: 0.9954 - val_loss: 0.0177 - val_accuracy: 0.9974
